In [1]:
#By David Basili

#THINGS FOR USER TO ADD

#put them as strings - for example: '/Users/dbasili/koger_drive/long-buffalo-data/positions.npy'
positions_path = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/video-annotations/test_extract/positions.npy' #the dots marking the animals on each frame
tracks_path = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/video-annotations/test_extract/tracks.npy'
picture_folder_path = '/media/golden/72FFC6EE48B5CF39/kenya-tracking/raw_frames/scare_clips/PZ/APR03_2018_C_DJI_0284/*jpg'

In [2]:
#SETUP

#import
import cv2
import numpy as np
import glob

#load global variables
listofpositions = np.load(positions_path) #positions.npy
listoftracks = np.load(tracks_path) #tracks.npy
files = glob.glob(picture_folder_path) #can load pictures from here
files.sort(key=lambda file: int(file.split('.')[-2].split('_')[-1])) #sort files
full_pic = np.zeros((580,1024,3), dtype=np.uint8) #image frame
framecount = 0 #for framecount
n = listoftracks[0]['first_frame'] + len(listoftracks[0]['track']) #to get good frame
if n >= len(files)/2:
    framecount = int(len(files)/2) - 1
else:
    framecount = n
trackcount = 1
focaltrackcount = 0
factor = 0.25

#TEMPORARILY, add ['connected'] to tracks - each entry marks which point number entry is beginning of a connected track
for track in listoftracks:
    track['connected'] = []

In [ ]:
#CLASSES

#draw points on image
def draw_points(picture, listofpositions, color1,color2,color3, r):
    for i in listofpositions:
        cv2.circle(picture, (int(i[1]), np.size(picture,0) - int(i[0])), r, (color1,color2,color3), -1)

#to process clicking on a button
def clicked(event, x, y, flags, param):
    global buttons, full_pic
    if event == cv2.EVENT_LBUTTONDOWN:
        for button in buttons:
            if button.check_if_pressed(x,y,full_pic):
                break
        else:
            add_point(x,y)

#to draw window
def draw_window():
    global factor, full_pic, files, framecount, trackcount, listoftracks, focaltrackcount
    
    image = cv2.imread(files[2*framecount]) #read image - skipping by 2s
    draw_points(image, listofpositions[framecount], 0,0,255, 10) #draw positions
    draw_points(image, listoftracks[focaltrackcount]['track'], 0,255,0, 5) # draw focal track
    draw_points(image, listoftracks[trackcount]['track'], 255,0,0, 5) # draw new track
    new_pic = cv2.resize(image, (int(np.size(image,1)*factor), int(np.size(image,0)*factor))) #width=1024 height=540
    full_pic[0:540,0:1024] = new_pic[0:540,0:1024] #put onto frame
    num = listoftracks[trackcount]['first_frame'] - listoftracks[focaltrackcount]['first_frame'] #to compute track
    num -= len(listoftracks[focaltrackcount]['track'])
    cv2.putText(full_pic, 'frames ahead: %d'%num, (820,20), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    cv2.imshow('pic0', full_pic)

# add a point to picture and to current track
def add_point(x, y):
    global trackcount, framecount, focaltrackcount, listoftracks
    x = 4*x #to adjust for picture
    y = 4*540 - 4*y #to adjust for picture
    listoftracks[focaltrackcount]['connected'].append(len(listoftracks[focaltrackcount]['track'])) #put marker
    listoftracks[focaltrackcount]['track'] = np.vstack((listoftracks[focaltrackcount]['track'], np.array([y,x]))) #add
    listofpositions[framecount] = np.vstack((listofpositions[framecount], np.array([y,x]))) #add to frame
    draw_window()

#find a good track to go to, given the focal track
def find_trackcount():
    global listoftracks, trackcount, focaltrackcount
    for i in range(len(listoftracks)): #make trackcount above 0
        num = listoftracks[i]['first_frame'] - listoftracks[focaltrackcount]['first_frame']
        num -= len(listoftracks[focaltrackcount]['track'])
        if num >= 0:
            trackcount = i
            break

#to call right function when button is pressed
def function_caller(function):
    if function == 'next_frame_function':
        next_frame_function()
    elif function == 'back_frame_function':
        back_frame_function()
    elif function == 'next_track_function':
        next_track_function()
    elif function == 'back_track_function':
        back_track_function()
    elif function == 'next_focal_track_function':
        next_focal_track_function()
    elif function == 'back_focal_track_function':
        back_focal_track_function()
    elif function == 'remove_point_function':
        remove_point_function()
    elif function == 'add_to_track_function':
        add_to_track_function()
    elif function == 'remove_track_function':
        remove_track_function()
    elif function == 'blue_function':
        blue_function()
    elif function == 'green_function':
        green_function()

#to help have keyboard shortcuts
def detect_keys(key):
    if key == 3: #right key = move frame forward
        next_frame_function()
    elif key == 2: #left key = move frame back
        back_frame_function()
    elif key == 0: #up key = move track forward
        next_track_function()
    elif key == 1: #down key = move track back
        back_track_function()
    elif key == ord('.'): # . key = move focal track forward
        next_focal_track_function()
    elif key == ord(','): # , key = move focal track back
        back_focal_track_function()
    elif key == 127: #delete key = delete point
        remove_point_function()
    elif key == 32: #space key = add to track
        add_to_track_function()
    elif key == 45: # - key = remove track
        remove_track_function()
    elif key == ord('g'): # 'g' key = green function
        green_function()
    elif key == ord('b'): # 'b' key = blue function
        blue_function()

#button class
class Button():
    def __init__(self,image,x1,x2,y1,y2,function,color1,color2,color3,text): #constructor
        self.x1 = x1
        self.x2 = x2
        self.y1 = y1
        self.y2 = y2
        self.color1 = color1
        self.color2 = color2
        self.color3 = color3
        self.text = text
        self.function = function
        image[y1:y2,x1:x2] = [color1,color2,color3]
        cv2.putText(image, text, (x1+5,y1+22), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    def check_if_pressed(self,x,y,image): #check if pressed button
        if x >= self.x1 and x <= self.x2 and y >= self.y1 and y <= self.y2: #if pressed
            #turn black
            image[self.y1:self.y2,self.x1:self.x2] = [0,0,0]
            cv2.imshow('pic0', full_pic)
            cv2.waitKey(200)
            
            #turn back
            image[self.y1:self.y2,self.x1:self.x2] = [self.color1,self.color2,self.color3]
            cv2.putText(image, self.text, (self.x1+5,self.y1+22), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,0),1,cv2.LINE_AA)
            cv2.imshow('pic0', full_pic)
            cv2.waitKey(1)
            
            #CALL FUNCTION
            function_caller(self.function)
            return True
        else:
            pass


#functions for buttons
def next_frame_function():
    global framecount, files
    n = framecount + 1
    if n < len(files)/2 and n >= 0:
        framecount = n
    draw_window()

def back_frame_function():
    global framecount
    n = framecount - 1
    if n < len(files)/2 and n >= 0:
        framecount = n
    draw_window()

def next_track_function():
    global trackcount, listoftracks
    n = trackcount + 1
    if n < len(listoftracks) and n >= 0:
        trackcount = n
    draw_window()

def back_track_function():
    global trackcount, listoftracks
    n = trackcount - 1
    if n < len(listoftracks) and n >= 0:
        trackcount = n
    draw_window()

def next_focal_track_function():
    global trackcount, focaltrackcount, framecount, listoftracks, files
    #check that doesn't go out of bounds
    n1 = focaltrackcount + 1
    if n1 < len(listoftracks) and n1 >= 0:
        focaltrackcount = n1
    #to get good frame
    n = listoftracks[focaltrackcount]['first_frame'] + len(listoftracks[focaltrackcount]['track'])
    if n >= len(files)/2:
        framecount = int(len(files)/2) - 1
    else:
        framecount = n
    find_trackcount() #make blue track start after green track ends
    draw_window()

def back_focal_track_function():
    global focaltrackcount, framecount, listoftracks, files, trackcount
    #check that doesn't go out of bounds
    n1 = focaltrackcount - 1
    if n1 < len(listoftracks) and n1 >= 0:
        focaltrackcount = n1
    #to get good frame
    n = listoftracks[focaltrackcount]['first_frame'] + len(listoftracks[focaltrackcount]['track'])
    if n >= len(files)/2:
        framecount = int(len(files)/2) - 1
    else:
        framecount = n
    find_trackcount() #make blue track start after green track ends
    draw_window()

def remove_point_function():
    global focaltrackcount, framecount
    the_track = listoftracks[focaltrackcount]['track'] #to make code cleaner
    if len(the_track) > 0: #make sure no exceptions
        listoftracks[focaltrackcount]['track'] = np.delete(the_track, len(the_track) - 1, 0)
    if len(listofpositions[framecount]) > 0: #for no exceptions
        listofpositions[framecount] = np.delete(listofpositions[framecount], len(listofpositions[framecount])-1, 0)
    draw_window()

def add_to_track_function():
    global trackcount, focaltrackcount, listoftracks
    if focaltrackcount == trackcount: #make sure isn't same as focal track
        pass
    else:
        listoftracks[focaltrackcount]['connected'].append(len(listoftracks[focaltrackcount]['track'])) #put marker
        temp = np.vstack((listoftracks[focaltrackcount]['track'], listoftracks[trackcount]['track'])) #for cleaner code
        listoftracks[focaltrackcount]['track'] = temp
        listoftracks = np.delete(listoftracks, trackcount, 0)
    find_trackcount() #make blue track start after green track ends
    draw_window()

def remove_track_function():
    global trackcount, listoftracks
    if len(listoftracks) > 0:
        listoftracks = np.delete(listoftracks, trackcount, 0)
    draw_window()

def blue_function():
    global trackcount, framecount, listoftracks, files
    n = listoftracks[trackcount]['first_frame'] #to get good frame
    if n >= len(files)/2:
        framecount = int(len(files)/2) - 1
    else:
        framecount = n
    draw_window()

def green_function():
    global focaltrackcount, framecount, listoftracks, files
    n = listoftracks[focaltrackcount]['first_frame'] + len(listoftracks[focaltrackcount]['track']) #to get good frame
    if n >= len(files)/2:
        framecount = int(len(files)/2) - 1
    else:
        framecount = n
    draw_window()

In [ ]:
#RUN

#buttons make frame first time
n = 9
back_frame_button = Button(full_pic,1,100,541,580,'back_frame_function',0,0,204,'frame -')
blue_button = Button(full_pic,101,127,541,560,'blue_function',204,0,0,' ')
green_button = Button(full_pic,101,127,561,580,'green_function',0,204,0,' ')
next_frame_button = Button(full_pic,128,int(2*1024/n),541,580,'next_frame_function',204,0,0,'frame +')
back_track_button = Button(full_pic,int(2*1024/n)+1,int(3*1024/n),541,580,'back_track_function',0,0,204,'track -')
next_track_button = Button(full_pic,int(3*1024/n)+1,int(4*1024/n),541,580,'next_track_function',204,0,0,'track +')
back_focal = Button(full_pic,int(4*1024/n)+1,int(5*1024/n),541,580,'back_focal_track_function',0,0,204,'focal track-')
next_focal = Button(full_pic,int(5*1024/n)+1,int(6*1024/n),541,580,'next_focal_track_function',204,0,0,'focal track+')
remove_point = Button(full_pic,int(6*1024/n)+1,int(7*1024/n),541,580,'remove_point_function',0,0,204,'remove point')
add_to_track = Button(full_pic,int(7*1024/n)+1,int(8*1024/n),541,580,'add_to_track_function',204,0,0,'add to track')
remove_track = Button(full_pic,int(8*1024/n)+1,int(9*1024/n),541,580,'remove_track_function',0,0,204,'remove track')

buttons = [back_frame_button,next_frame_button,back_track_button,next_track_button,back_focal] #buttons
buttons.extend([next_focal,remove_point,add_to_track,remove_track,blue_button, green_button])

#get good track, draw image on window, set mouse to work
find_trackcount()
draw_window()
cv2.namedWindow('pic0')
cv2.setMouseCallback('pic0', clicked)

#loop to keep image updating
while True:
    cv2.imshow('pic0', full_pic)
    #stop loop if press 'x'
    key = cv2.waitKey(2) &0xff
    detect_keys(key)
    if key == 27:
        break
cv2.destroyAllWindows()